In [0]:
import requests
import pandas
import pprint
from IPython.display import clear_output

In [0]:
parent_path = ''

In [0]:
def fetch_data(page=None, quality='A'):
    query_params = {"cnt": 'brazil',
                    "q": quality,
                    "type": "song",
#                     "also": ""
    #                 "length|": "0:47"
    #                 "gen": genus,
    #                 "rec": recordist,
    #                 "loc": f'"{location}"' if location else None,  # Location must be wrapped in double quotes.
    #                 "rmk": remarks,
    #                 "lat": latitude,
    #                 "lon": longitude,
    #                 "box": box,
    #                 "nr": catalogue_number,
    #                 "lic": license,
    #                 "area": area,
    #                 "since": since,
    #                 "year": year,
    #                 "month": month
                   }
    query = " ".join([f"{name}:{var}" for name, var in query_params.items() if var])
    r = requests.get('https://www.xeno-canto.org/api/2/recordings', params={"query": query, "page": page})
    return r.json()

In [0]:
def append_to_frame(data_frame, recordings):
    for recording in recordings['recordings']:
        # print(recording)
        mins_secs = recording['length'].split(':')
        data_frame = data_frame.append({
            'file_id': recording['id'],
            'genus': recording['gen'],
            'species': recording['sp'],
            'name': recording['en'],
            'download_url': 'https://www.xeno-canto.org/{}/download'.format(recording['id']),
            'file_name': recording['file-name'],
            'duration': int(mins_secs[0]) * 60 + int(mins_secs[1]),
            'quality': recording['q'],
            'other_species_in_recording': ', '.join(recording['also'])
        }, ignore_index=True)
    return data_frame

In [0]:
birds = pandas.DataFrame(columns=['file_id', 'genus', 'species', 'name', 'download_url', 'file_name', 'duration', 'other_species_in_recording', 'quality'])

In [0]:
birds.head()

,file_id,genus,species,name,download_url,file_name,duration,other_species_in_recording,quality


In [0]:
for q in ['A', 'B']:
    initial_data = fetch_data(quality=q)
    num_pages = initial_data['numPages']
    num_recordings = initial_data['numRecordings']
    birds = append_to_frame(birds, initial_data)
    for i in range(2, num_pages+1, 1):
        # clear_output(wait=True)
        birds = append_to_frame(birds, fetch_data(page=i, quality=q))
        print('Fetched {} of {} pages of quality {} songs'.format(i, num_pages, q))

Fetched 2 of 23 pages of quality A songs
Fetched 3 of 23 pages of quality A songs
Fetched 4 of 23 pages of quality A songs
Fetched 5 of 23 pages of quality A songs
Fetched 6 of 23 pages of quality A songs
Fetched 7 of 23 pages of quality A songs
Fetched 8 of 23 pages of quality A songs
Fetched 9 of 23 pages of quality A songs
Fetched 10 of 23 pages of quality A songs
Fetched 11 of 23 pages of quality A songs
Fetched 12 of 23 pages of quality A songs
Fetched 13 of 23 pages of quality A songs
Fetched 14 of 23 pages of quality A songs
Fetched 15 of 23 pages of quality A songs
Fetched 16 of 23 pages of quality A songs
Fetched 17 of 23 pages of quality A songs
Fetched 18 of 23 pages of quality A songs
Fetched 19 of 23 pages of quality A songs
Fetched 20 of 23 pages of quality A songs
Fetched 21 of 23 pages of quality A songs
Fetched 22 of 23 pages of quality A songs
Fetched 23 of 23 pages of quality A songs
Fetched 2 of 26 pages of quality B songs
Fetched 3 of 26 pages of quality B songs
Fe

In [0]:
birds.to_csv(parent_path+'birds_dataset_unfiltered.csv')

In [0]:
birds = birds[birds['duration'] >= 20][birds['name'] != 'Identity unknown']
birds = birds[birds.groupby('genus').genus.transform(len) > 25].reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [0]:
birds.to_csv(parent_path+'birds_dataset.csv')